In [3]:
import logging
import pdb
logging.basicConfig(level=logging.INFO)
%matplotlib inline
%run LocalRepo.ipynb
%run repos.ipynb
%run parsing.ipynb
%run metrics.ipynb
%run analysis.ipynb
import pyfiglet
import numpy as np
# np.seterr(all='raise')
import os


repos = [
    "ErikBrendel/LudumDare", # 8 minutes BTM
    "eclipse/eclipse.jdt.core",  # from duerschmidt
    "jenkinsci/jenkins",
    "wumpz/jhotdraw",
    "neuland/jade4j",
    "jfree/jfreechart",
    "brettwooldridge/HikariCP",  # 1.2h BTM
    "adamfisk/LittleProxy",  # 1.5h BTM
    "dynjs/dynjs",  # 2.5h BTM
    # "SonarSource/sonarqube",  # quite big / prob. several hours BTM
    "square/okhttp",
    "eclipse/che",
    "elastic/elasticsearch",
    "apache/camel",
    "jOOQ/jOOQ",
]
repos = ["wumpz/jhotdraw"]



for repo in repos:
    r = LocalRepo(repo)
    print(pyfiglet.figlet_format(r.name))
    r.update()
    
    # pdb.set_trace()
    print("Nodes in project structure:", r.get_tree().node_count())
    
    for view in []: # ["structural", "evolutionary", "linguistic"]
        # MetricManager.clear(r, view)

        coupling_graph = MetricManager.get(r, view)
        # coupling_graph.plaintext_save(repo)
        # coupling_graph.html_save(repo)
        

        print("\nResults from the " + view + " view:\n")

        coupling_graph.print_statistics()
        coupling_graph.print_most_linked_nodes()
        # coupling_graph.show_weight_histogram()
        # coupling_graph.visualize(use_spring=False, with_labels=False)
        # coupling_graph.visualize(use_spring=True, with_labels=False)
        print("\n")
    # continue
    
    # amalysis
    repo_tree = r.get_tree()
    def node_filter(node_path):
        tree_node = repo_tree.find_node(node_path)
        return tree_node.get_type() == "method" and tree_node.get_line_span() >= 5
    def node_pair_filter(node_a_path, node_b_path):
        return True #path_module_distance(node_a_path, node_b_path) >= 0
        
    #analyze_disagreements(r, ["structural", "evolutionary", "linguistic"], [
    #    [0, 1, None, "unrelated functionality that gets adjusted together"],
    #    [0, 0, 1, "duplicated concepts, developed separately"],
    #    [None, 0, 1, "duplicated concepts, adjusted separately"],
    #    [1, None, 0, "weakly modularized code"]
    #], node_filter, node_pair_filter)
    
    """
    analyze_disagreements(r, ["structural", "evolutionary", "module_distance"], [
        [1, 0, None, "using each other, but not developed together"],
        [0, 1, None, "unrelated functionality that gets adjusted together"],
        [1, None, 0, "Strong structural coupling across a big distance"],
        [None, 1, 0, "Strong evolutionary coupling across a big distance"],
        [0, 1, 0, "Strong evolutionary and low structural coupling across a big distance"],
    ], node_filter, node_pair_filter)
    """
    analyze_disagreements(r, ["structural", "evolutionary", "linguistic", "module_distance"], [
        [None, None, 1, None, "Strongest ling"],
        [None, None, 0, None, "Weakest ling"],
        [1, 0, 0, 0, "Strong structural and low evolutionary / linguistic coupling across a big distance"],
        [0, 1, 0, 0, "Strong evolutionary and low structural / linguistic coupling across a big distance"],
        [0, 0, 1, 0, "Strong linguistic and low structural / evolutionary coupling across a big distance"],
        [0, 0, 0, 1, "Close but totally unrelated"],
        [0, 1, 1, 0, "Evo + Ling but no struct + big distance"],
    ], node_filter, node_pair_filter)
    
    #analyze_disagreements(r, ["evolutionary", "module_distance"], [
    #    [1, 0, "Strong evolutionary coupling across a big distance"],
    #], node_filter, node_pair_filter)
    
    #analyze_disagreements(r, ["structural", "evolutionary"], [
    #    [0, 1, "adjusted together without structural relation"],
    #], node_filter, node_pair_filter)
    
print("\nProgram is over!")

                                      ___ _           _      _           
__      ___   _ _ __ ___  _ __ ____  / (_) |__   ___ | |_ __| |_ __ __ _ 
\ \ /\ / / | | | '_ ` _ \| '_ \_  / / /| | '_ \ / _ \| __/ _` | '__/ _` |
 \ V  V /| |_| | | | | | | |_) / / / / | | | | | (_) | || (_| | | | (_| |
  \_/\_/  \__,_|_| |_| |_| .__/___/_/ _/ |_| |_|\___/ \__\__,_|_|  \__,_|
                         |_|         |__/                                
          
__      __
\ \ /\ / /
 \ V  V / 
  \_/\_/  
          

updating wumpz/jhotdraw...
Repo is up to date!
Analyzing 669 of 671 files
Skipped 2 test roots: ['jhotdraw-utils/src/test', 'jhotdraw-core/src/test']
Found 11208 classes, methods and fields!
Nodes in project structure: 11208
Using precalculated structural values


Propagating down coupling information, iteration 1/2:   0%|          | 0/1254 [00:00<?, ?it/s]

Applying changes, iteration 1/2:   0%|          | 0/762283 [00:00<?, ?it/s]

Propagating down coupling information, iteration 2/2:   0%|          | 0/1254 [00:00<?, ?it/s]

Applying changes, iteration 2/2:   0%|          | 0/904461 [00:00<?, ?it/s]

...


Pattern 5 ([0, 0, 0, 1, 'Close but totally unrelated']):


interactive(children=(FloatLogSlider(value=0.25, description='structural', layout=Layout(width='500px'), max=0…


Pattern 6 ([0, 1, 1, 0, 'Evo + Ling but no struct + big distance']):


interactive(children=(FloatLogSlider(value=0.25, description='structural', layout=Layout(width='500px'), max=0…


Program is over!
